In [96]:
import sim
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [97]:
sim.simxFinish(-1)  # Close opened connections
clientID = sim.simxStart('127.0.0.1', 19999, True, True, 5000, 5)  # Connect to CoppeliaSim

if clientID != -1:
    print('Connected')

    # Now try to retrieve data in a blocking fashion (i.e. a service call):
    res, objs = sim.simxGetObjects(clientID, sim.sim_handle_all, sim.simx_opmode_blocking)

    print('Simulation time in milliseconds: ', sim.simxGetLastCmdTime(clientID))
    
    # Get Object position
    name = 'Omnirob'
    err_code, cuboid = sim.simxGetObjectHandle(clientID, name, sim.simx_opmode_blocking)
    res, position = sim.simxGetObjectPosition(clientID, cuboid, -1, sim.simx_opmode_blocking)        
    print('Omnirob is at [x,y,z]=', position)
    
    # Now close the connection to CoppeliaSim:
    sim.simxGetPingTime(clientID)
    sim.simxFinish(clientID)
    print('Disconnected')
else:
    print('Failed connecting to remote API server')

Connected
Simulation time in milliseconds:  36750
Omnirob is at [x,y,z]= [7.624996185302734, 8.547089576721191, 0.3544994592666626]
Disconnected


In [98]:
class robot():

    def __init__(self, frame_name, motor_names=[], client_id=0):
        # If there is an existing connection
        if client_id:
                self.client_id = client_id
        else:
            self.client_id = self.open_connection()

        self.motors = self._get_handlers(motor_names)

        # Robot frame
        self.frame =  self._get_handler(frame_name)

    def open_connection(self):
        sim.simxFinish(-1)  # just in case, close all opened connections
        self.client_id = sim.simxStart('127.0.0.1', 19999, True, True, 5000, 5)  # Connect to CoppeliaSim

        if clientID != -1:
            print('Robot connected')
        else:
            print('Connection failed')
        return clientID

    def close_connection(self):
        # Before closing the connection to CoppeliaSim, make sure that the last command sent out had time to arrive.
        sim.simxGetPingTime(self.client_id)
        # Now close the connection to CoppeliaSim:
        sim.simxFinish(self.client_id)
        print('Connection closed')

    def isConnected(self):
        c,result = sim.simxGetPingTime(self.client_id)
        # Return true if the robot is connected
        return result > 0

    def _get_handler(self, name):
        err_code, handler = sim.simxGetObjectHandle(self.client_id, name, sim.simx_opmode_blocking)
        return handler

    def _get_handlers(self, names):
        handlers = []
        for name in names:
            handler = self._get_handler(name)
            handlers.append(handler)

        return handlers

    def send_motor_velocities(self, vels):
        for motor, vel in zip(self.motors, vels):
            err_code = sim.simxSetJointTargetVelocity(self.client_id,
                                                      motor, vel, sim.simx_opmode_streaming)

    def set_position(self, position, relative_object=-1):
        if relative_object != -1:
            relative_object = self._get_handler(relative_object)
        sim.simxSetObjectPosition(clientID, self.frame, relative_object, position, sim.simx_opmode_oneshot)

    def simtime(self):
        return sim.simxGetLastCmdTime(self.client_id)

    def get_position(self, relative_object=-1):
        # Get position relative to an object, -1 for global frame
        if relative_object != -1:
            relative_object = self._get_handler(relative_object)
        res, position = sim.simxGetObjectPosition(self.client_id, self.frame, relative_object, sim.simx_opmode_blocking)
        return array(position)

    def get_object_position(self, object_name):
        # Get Object position in the world frame
        err_code, object_h = sim.simxGetObjectHandle(self.client_id, object_name, sim.simx_opmode_blocking)
        res, position = sim.simxGetObjectPosition(self.client_id, object_h, -1, sim.simx_opmode_blocking)
        return array(position)

    def get_object_relative_position(self, object_name):
        # Get Object position in the robot frame
        err_code, object_h = sim.simxGetObjectHandle(self.client_id, object_name, sim.simx_opmode_blocking)
        res, position = sim.simxGetObjectPosition(self.client_id, object_h, self.frame, sim.simx_opmode_blocking)
        return array(position)

In [99]:
def move_robot(waypoint_name, motor_names, robot_name, tf):
    # Joint name in coppelia
    r = robot(robot_name, motor_names)  # Create an instance of our robot

    # Initial locations
    robot_position = r.get_position()
    print('Robot position: (%.2f, %.2f) '%(robot_position[0], robot_position[1]))

    desired_position = r.get_object_position(waypoint_name)
    print('Desired position: (%.2f, %.2f) '%(desired_position[0], desired_position[1]))

    relative_position = r.get_object_relative_position(waypoint_name)
    print('Relative position: (%.2f, %.2f) '%(relative_position[0], relative_position[1]))


    # Virtual disc
    disc = robot('Disc', client_id=r.client_id)

    # Time interval
    time_steps = linspace(0, tf, 500)

    a1 = (desired_position - robot_position) / tf
    a0 = robot_position

    for t in time_steps:
        point_traj = a1 * t + a0
        vel_traj = a1
    #    print(point_traj)

        disc.set_position(point_traj)


        # Sensing
        robot_position = r.get_position()


        # Trajectory tracker
        u = 10 * (point_traj - robot_position) + vel_traj

        vx, vy, vz = u
        r.send_motor_velocities([-vy - vx, vy - vx, vy + vx, -vy + vx])



        time.sleep(tf/500)

    r.send_motor_velocities([0, 0, 0, 0])
    r.close_connection()

In [100]:
motor_names = ['Omnirob_FLwheel_motor', 'Omnirob_FRwheel_motor', 'Omnirob_RRwheel_motor', 'Omnirob_RLwheel_motor']
robot_name = 'Omnirob'
waypoint_name = 'Sphere1'
move_robot(waypoint_name, motor_names, robot_name, 3)
waypoint_name = 'Disc0'
move_robot(waypoint_name, motor_names, robot_name, 1)
waypoint_name = 'Disc1'
move_robot(waypoint_name, motor_names, robot_name, 1)
waypoint_name = 'Sphere'
move_robot(waypoint_name, motor_names, robot_name, 1)
waypoint_name = 'Disc2'
move_robot(waypoint_name, motor_names, robot_name, 2)
waypoint_name = 'Sphere0'
move_robot(waypoint_name, motor_names, robot_name, 1)
waypoint_name = 'Disc3'
move_robot(waypoint_name, motor_names, robot_name, 3)
waypoint_name = 'Sphere2'
move_robot(waypoint_name, motor_names, robot_name, 3)
waypoint_name = 'Disc4'
move_robot(waypoint_name, motor_names, robot_name, 1)
waypoint_name = 'Disc5'
move_robot(waypoint_name, motor_names, robot_name, 1)
waypoint_name = 'Sphere3'
move_robot(waypoint_name, motor_names, robot_name, 2)
waypoint_name = 'Disc6'
move_robot(waypoint_name, motor_names, robot_name, 2)
waypoint_name = 'Disc3'
move_robot(waypoint_name, motor_names, robot_name, 1)
waypoint_name = 'Disc7'
move_robot(waypoint_name, motor_names, robot_name, 2)
waypoint_name = 'Sphere4'
move_robot(waypoint_name, motor_names, robot_name, 1)

Robot connected
Robot position: (7.62, 8.53) 
Desired position: (0.40, 8.05) 
Relative position: (-7.22, -0.48) 
Robot connected
Connection closed
Robot connected
Robot position: (0.39, 8.15) 
Desired position: (4.05, 7.12) 
Relative position: (3.80, 0.28) 
Robot connected
Connection closed
Robot connected
Robot position: (4.29, 7.02) 
Desired position: (4.40, 3.50) 
Relative position: (0.99, -3.38) 
Robot connected
Connection closed
Robot connected
Robot position: (4.38, 3.27) 
Desired position: (1.93, 2.22) 
Relative position: (-2.24, -1.45) 
Robot connected
Connection closed
Robot connected
Robot position: (1.77, 2.21) 
Desired position: (-2.48, 5.73) 
Relative position: (-4.98, 2.37) 
Robot connected
Connection closed
Robot connected
Robot position: (-2.51, 5.86) 
Desired position: (-3.57, 3.40) 
Relative position: (0.03, -2.68) 
Robot connected
Connection closed
Robot connected
Robot position: (-3.69, 3.24) 
Desired position: (-1.98, -2.55) 
Relative position: (3.85, -4.66) 
Robot